<a href="https://colab.research.google.com/github/jeetu-g2-1/My_Projects/blob/main/Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

In [ ]:
def get_cities(state):
  text = requests.get(base+'/'+state).content
  soup = BeautifulSoup(text, 'html.parser')
  cities = soup.find_all('a', class_ = 'destTitle')
  result = []
  for city in cities[:-8]:
    result.append(city.text.split('.')[1][1:])
  return result

In [ ]:
def get_place_details(place):
  base = 'https://www.trawell.in'
  url = base + place
  text = requests.get(url).content
  soup = BeautifulSoup(text, 'html.parser')
  data = {}
  data['place'] = soup.find('h2').text[6:]
  spans = soup.find('div', id = 'quick-info').find_all('span')
  data['country'] = spans[1].text
  data['state'] = spans[4].text
  data['trip duration'] = spans[9].text
  data['base station'] = spans[11].text
  data['best time'] = spans[15].text
  h3s = soup.find_all('h3', class_='destTitle')
  l = len(h3s)
  best = []
  for i in range(l//2):
    best.append(h3s[i].find('a').text)
  data['best places'] = best
  # via
  travel_by = []
  by = soup.find_all('div', class_='destTransportHead')[0]
  h3s = by.find_all('h3')
  for h3 in h3s[: -1]:
    travel_by.append(h3.text.split(' ')[1])
  data['travel by'] = travel_by
  return data

In [ ]:
base = 'https://www.trawell.in'
get_place_details('/andhra/araku-valley')

{'place': 'Araku Valley',
 'country': 'India',
 'state': 'Andhra',
 'trip duration': '1-2 Days',
 'base station': 'Araku Valley',
 'best time': 'September to May',
 'best places': ['Borra Caves',
  'Ananthagiri',
  'Chaparai / Chaaparai',
  'Tribal Museum',
  'Katiki / Katika Waterfalls'],
 'travel by': ['Air', 'Train', 'Bus']}

In [ ]:
def check_word(word):
  for char in word:
    if not char.isalpha() | char.isspace():
      return False
  return True

In [ ]:
def get_data(state):
  base = 'https://www.trawell.in'
  text = requests.get(base+'/'+state).content
  soup = BeautifulSoup(text, 'html.parser')
  cities = soup.find_all('a', class_ = 'destTitle')
  result = []
  for city in cities[:-8]:
    result.append(city.text.split('.')[1][1:])
  result = [word.lower() for word in result]
  result = [word for word in result if check_word(word)]
  result = [word.replace(' ','-') for word in result]
  l = len(result)
  data = []
  for i in range(l):
    try:
      data.append(get_place_details(state+'/'+result[i]))
    except AttributeError as e:
      continue
  return data

In [ ]:
import pandas as pd

In [ ]:
states = ['/andhra', '/tamilnadu', '/kerala', '/karnataka', '/goa', '/maharashtra', '/gujarat', '/rajasthan']

In [ ]:
total_data = []

for state in states:
  total_data.extend(get_data(state))

In [ ]:
df = pd.DataFrame(total_data)
df

,place,country,state,trip duration,base station,best time,best places,travel by
0,Vizag,India,Andhra,2-3 Days,Vizag,November to February,"[Kailasagiri, Rama Krishna Beach, Rishikonda B...","[Air, Train, Bus]"
1,Araku Valley,India,Andhra,1-2 Days,Araku Valley,September to May,"[Borra Caves, Ananthagiri, Chaparai / Chaapara...","[Air, Train, Bus]"
2,Tirupati,India,Andhra,1-2 Days,Tirupati,October to March,"[Tirumala, Govindaraja Swami Temple, Kapila Th...","[Air, Train, Bus]"
3,Srisailam,India,Andhra,1-2 Days,Srisailam,September to March,"[Mallikarjuna Swamy Temple, Bramarambha Temple...","[Air, Train, Bus]"
4,Papikondalu,India,Andhra,1-2 Days,Rajahmundry (60 Kms),October to January,[Perantala Palli],"[Air, Train, Bus]"
...,...,...,...,...,...,...,...,...
157,Bikaner,India,Rajasthan,1-2 Days,Bikaner,October to March,"[Junagarh Fort, Lalgarh Palace & Museum, Bhand...","[Air, Train, Bus]"
158,Pushkar,India,Rajasthan,1-2 Days,Pushkar,October to March,"[Brahma Temple, Pushkar Lake, Varaha Temple, P...","[Air, Train, Bus]"
159,Kumbhalgarh,India,Rajasthan,Full Day,"Kumbhalgarh, Udaipur (89 Kms)",October to March,"[Kumbhalgarh Fort, Ranakpur Jain Temples]","[Air, Train, Bus]"
160,Alwar,India,Rajasthan,1-2 Days,Alwar,September to February,"[Bala Quila / Alwar Fort, City Palace, Siliser...","[Air, Train, Bus]"


In [ ]:
df.shape

(162, 8)

In [ ]:
df.to_csv('data.csv')